## OpenAPI 활용

1. https://data.go.kr 로그인
2. 기상청 종관 ASOS 일별 관측자료 API 신청


In [2]:
## read_csv
import pandas as pd

In [4]:
df = pd.read_csv('../day01/data/충북푸드뱅크현황.csv', encoding='utf-8')
df.tail()

,지역,시군구,센터단위,센터구분,지원센터,신고구분,운영단체명,운영법인분류(대),대표자명,센터전화번호,주소,등록일자
24,충북,충청북도 음성군,기초,푸드뱅크,한울사랑나눔,당연신고,한울사랑나눔푸드뱅크,개인 운영,이*경,043-878-4312,음성군 생극면 음성로 1620번길 7,2011-11-25
25,충북,충청북도 음성군,기초,푸드뱅크,관성푸드뱅크,당연신고,개인,개인 운영,유*호,043-877-9457,음성군 삼성면 대성로547번길 12,2014-04-18
26,충북,충청북도 단양군,기초,푸드뱅크,단양군푸드뱅크,임의신고,단양군사회복지협의회,법인 운영,김*학,043-422-1299,"단양군 단양읍 수변로 83, 3층",2022-12-01
27,충북,충청북도 단양군,기초,푸드뱅크,단양나눔과기쁨,당연신고,기독교대한성결교회(단양수정교회),법인 운영,함*용,043-421-3191,단양군 대강면 장림1길 6,2012-04-17
28,충북,충청북도 단양군,기초,푸드마켓,단양군푸드마켓,임의신고,단양군사회복지협의회,법인 운영,김*학,043-422-1700,"단양군 단양읍 중앙1로 29, 1층",2012-06-22


#### 오픈API
- 기상총 종관 ASOS OpenAPI 신청건 웹브라우저 테스트
- 테스트 성공한 URL
    - https://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=JzmUY2JqiPqaZHmZ7VDke8wMFu3m%2FCXZSUCawmglK99g1cw5ytYYWZ%2F4VmiJz2Wn5MB1aBEA7N0YlXlJz%2B%2FK8A%3D%3D&pageNo=1&numOfRows=10&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt=20141001&endDt=20141030&stnIds=108

- urllib.request, parse 모듈 사용 파이썬에서 OPENAPI 호출

In [5]:
# openAPI 호출용 모듈
import requests
import json

#### Data 포탈 OpenAPI 두 가지 문제
- URL encoding을 사용해야 함 - URL결과를 복사해서 사용하면 인코딩 문제가 발생
    - params 딕셔너리로 작업할 때는 URL디코딩 사용
    - URL을 하나의 문자열로 그냥 쓸때는 URL인코딩 사용
- https는 사용하면 안됨. http 프로토콜 -> https로 자동 변경

In [17]:
## 부산한정 ASOS 종관데이터 조회 함수
def getAsosDailyData(startdate, enddate):
    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList' # endpoint url
    params = {'serviceKey': f'JzmUY2JqiPqaZHmZ7VDke8wMFu3m/CXZSUCawmglK99g1cw5ytYYWZ/4VmiJz2Wn5MB1aBEA7N0YlXlJz+/K8A==', #서비스키(디코딩)
              'pageNo': '1',  # 페이지번호
              'numOfRows': '10', # 한페이지 결과수
              'dataType': 'JSON',  # 응답자료형식 JSON / XML
              'dataCd': 'ASOS', # ASOS 고정
              'dateCd': 'DAY', # 날짜코드
              'startDt': startdate, # 조회시작일
              'endDt': enddate, # 조회종료일(D-1)
              'stnIds': '159'} # 종관기상관측 지점 번호(108 서울, 159 부산)

    response = requests.get(url, params=params) # GET으로 Request를 요청
    result = json.loads(response.text) # 문자열 결과를 json형태로 파싱
    items = result['response']['body']['items']['item']

    return items

In [19]:
res = getAsosDailyData('20241001', '20241010')

In [22]:
# list to Pandas Dataframe
df_asos_busan = pd.DataFrame(res)

In [24]:
df_asos_busan.tail(2)

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
8,159,부산,2024-10-09,19.3,16.6,2400,23.9,1145,,,...,22.7,24.5,25.0,22.1,20.4,3.0,4.3,0.0,,
9,159,부산,2024-10-10,19.8,16.6,0007,25.6,1301,0.0,,...,22.4,24.4,24.9,22.1,20.4,2.9,4.2,0.0,{무지개}0632-0725. {비}0714-0818. {비}1023-1034.,


In [25]:
df_asos_busan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   stnId           10 non-null     object
 1   stnNm           10 non-null     object
 2   tm              10 non-null     object
 3   avgTa           10 non-null     object
 4   minTa           10 non-null     object
 5   minTaHrmt       10 non-null     object
 6   maxTa           10 non-null     object
 7   maxTaHrmt       10 non-null     object
 8   mi10MaxRn       10 non-null     object
 9   mi10MaxRnHrmt   10 non-null     object
 10  hr1MaxRn        10 non-null     object
 11  hr1MaxRnHrmt    10 non-null     object
 12  sumRnDur        10 non-null     object
 13  sumRn           10 non-null     object
 14  maxInsWs        10 non-null     object
 15  maxInsWsWd      10 non-null     object
 16  maxInsWsHrmt    10 non-null     object
 17  maxWs           10 non-null     object
 18  maxWsWd      

#### 결론
- 데이터포털 OpenAPI로 가져오는 데이터에는 Null 데이터가 없음
- 데이터 수집 중 가장 편하게 사용할 수 있는 서비스 - OpenAPI
- 네이버API, 카카오API, 구글API, IMDB API등 전세계적으로 제공하고 있음
    - API 서비스 마다 사용법이 전부 달라서 숙지해야 